# Modeling (Clustering Unit)
## Corey Solitaire
#### 10.15.2020

In [1]:
import acquire
import prepare
import wrangle
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.float_format = '{:20,.2f}'.format
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression, SelectKBest, RFE 
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [2]:
#df = wrangle.get_zillow_data()
train, validate, test = wrangle.clean_zillow(wrangle.get_zillow_data()) 
train.shape, validate.shape, test.shape

((43332, 21), (18572, 21), (15476, 21))

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43332 entries, 14505 to 54422
Data columns (total 21 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   bathroomcnt                   43332 non-null  int64   
 1   bedroomcnt                    43332 non-null  int64   
 2   calculatedfinishedsquarefeet  43332 non-null  int64   
 3   latitude                      43332 non-null  float64 
 4   longitude                     43332 non-null  float64 
 5   lotsizesquarefeet             43332 non-null  int64   
 6   rawcensustractandblock        43332 non-null  int64   
 7   regionidzip                   43332 non-null  category
 8   roomcnt                       43332 non-null  int64   
 9   unitcnt                       43332 non-null  int64   
 10  yearbuilt                     43332 non-null  int64   
 11  structuretaxvaluedollarcnt    43332 non-null  float64 
 12  taxvaluedollarcnt             43332 non-nu

#### New Dataframes based on County

In [4]:
# LA County
la_df = train[train.county=='Los Angeles']
la_df.shape

# Ventura County
vc_df = train[train.county=='Ventura']
vc_df.shape

# Orange County
oc_df = train[train.county=='Orange']

la_df.shape, vc_df.shape, oc_df.shape

((28462, 21), (3452, 21), (11418, 21))

#### Remove Outliers from  New Dataframes using Z-Score

In [5]:
from scipy import stats
import numpy as np

# Must drop categorical varaibles, it turn in to cat codes before modeling
la_df.drop(columns=["regionidzip","county","propertylandusedesc","heatingorsystemdesc"], inplace = True)

# Drop Outliers in Dataframe (Set = 3 not 1.5)
# IQR of 1.5 and 3 removed too many values, so left at 6, and still removed 5,000 Properties

Q1 = la_df.quantile(0.25)
Q3 = la_df.quantile(0.75)
IQR = Q3 - Q1

la_df_out = la_df[~((la_df < (Q1 - 6 * IQR)) |(la_df > (Q3 + 6 * IQR))).any(axis=1)]
la_df_out.shape

(23029, 17)

***

#### Zip Codes per County

In [6]:
#Zip Codes LA

len(la_df['regionidzip'].unique()) 

KeyError: 'regionidzip'

Does LA have 280 Zipcodes? - Internet says yes, wow!

In [ ]:
#Zip Codes Ventura

len(vc_df['regionidzip'].unique()) 

In [ ]:
#Zip Codes Orange

len(oc_df['regionidzip'].unique()) 

***

#### Modeling 